In [9]:
import pretrainedmodels
import pretrainedmodels.utils as utils
import torch
import preprocessing

# Define device for torch
use_cuda = True
print("CUDA is available:", torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")


print(pretrainedmodels.model_names)

#Check pretrained settings

print(pretrainedmodels.pretrained_settings['vgg16_bn'])
# print(pretrainedmodels.pretrained_settings['vgg16'])
# print(pretrainedmodels.pretrained_settings['vgg19_bn'])
# print(pretrainedmodels.pretrained_settings['vgg19'])
# print(pretrainedmodels.pretrained_settings['resnet101'])
# print(pretrainedmodels.pretrained_settings['resnet152'])
# print(pretrainedmodels.pretrained_settings['resnet50'])

#Download pretrained models
model1 = pretrainedmodels.__dict__['vgg16_bn'](num_classes=1000, pretrained='imagenet')
# model2 = pretrainedmodels.__dict__['vgg16'](num_classes=1000, pretrained='imagenet')
# model3 = pretrainedmodels.__dict__['vgg19_bn'](num_classes=1000, pretrained='imagenet')
# model4 = pretrainedmodels.__dict__['vgg19'](num_classes=1000, pretrained='imagenet')
# model5 = pretrainedmodels.__dict__['resnet101'](num_classes=1000, pretrained='imagenet')
# model6 = pretrainedmodels.__dict__['resnet152'](num_classes=1000, pretrained='imagenet')
# model7 = pretrainedmodels.__dict__['resnet50'](num_classes=1000, pretrained='imagenet')
model1 = model1.to(device)
model1.eval()

CUDA is available: True
['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']
{'imagenet': {'url': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth', 'input_space': 'RGB', 'input_size': [3, 224, 224], 'input_range': [0, 1], 'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225], 'num_classes': 1000}}


/home/dbwchaos/group_19_tpdl/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dbwchaos/group_19_tpdl/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (avgpool): AdaptiveAvgPool2d(output_size=(7, 7))
  (_features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, di

In [3]:
# load_img = utils.LoadImage()

# # transformations depending on the model
# # rescale, center crop, normalize, and others (ex: ToBGR, ToRange255)
# tf_img = utils.TransformImage(model1) 

# path_img = 'data/mel/chinese/chinese0.npy'

# input_img = load_img(path_img)
# input_tensor = tf_img(input_img)         # 3x400x225 -> 3x299x299 size may differ
# input_tensor = input_tensor.unsqueeze(0) # 3x299x299 -> 1x3x299x299
# input = torch.autograd.Variable(input_tensor,
#     requires_grad=False)

# output_logits = model1(input) # 1x1000

In [ ]:
from torch.autograd import Variable
import os
import torch
import numpy as np
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

class MelSpectrogramDataset(Dataset):
    def __init__(self, root_dir, file_paths=None, labels=None):
        self.root_dir = root_dir
        self.data = []
        self.language_to_idx = {}

        # Locate the mel directory
        mel_dir = os.path.join(self.root_dir, 'mel')
        
        # Get the sorted list of language directories
        languages = sorted([
            lang for lang in os.listdir(mel_dir)
            if os.path.isdir(os.path.join(mel_dir, lang))
        ])

        # Create mapping from language to index
        self.language_to_idx = {lang: idx for idx, lang in enumerate(languages)}

        if file_paths is None or labels is None:
            # Store all (file_path, one-hot label) pairs if not provided
            for lang in languages:
                lang_dir = os.path.join(mel_dir, lang)
                for file_name in os.listdir(lang_dir):
                    if file_name.endswith('.npy'):
                        file_path = os.path.join(lang_dir, file_name)
                        # Create one-hot encoded label
                        label = torch.zeros(len(self.language_to_idx))
                        label[self.language_to_idx[lang]] = 1.0
                        self.data.append((file_path, label))
        else:
            # Use provided file paths and one-hot labels
            for file_path, label in zip(file_paths, labels):
                self.data.append((file_path, label))

        self.num_classes = len(self.language_to_idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file_path, one_hot_label = self.data[idx]
        mel_tensor = torch.tensor(np.load(file_path), dtype=torch.float32)

        return one_hot_label, mel_tensor

# Function to perform train-test split
def create_train_test_split(dataset, test_size=0.2):
    file_paths = [x[0] for x in dataset.data]
    labels = [x[1] for x in dataset.data]

    # Split the dataset
    train_file_paths, test_file_paths, train_labels, test_labels = train_test_split(
        file_paths, labels, test_size=test_size, random_state=42
    )

    # Create new datasets for train and test
    train_dataset = MelSpectrogramDataset(root_dir=dataset.root_dir, file_paths=train_file_paths, labels=train_labels)
    test_dataset = MelSpectrogramDataset(root_dir=dataset.root_dir, file_paths=test_file_paths, labels=test_labels)

    return train_dataset, test_dataset

# Example Usage
dataset = MelSpectrogramDataset(root_dir='data')
train_dataset, test_dataset = create_train_test_split(dataset, test_size=0.2)

# Creating DataLoaders for train and test datasets
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Example of using the train dataloader
for label, mel in train_dataloader:
    print("Train Label (one-hot):", label)
    print("Train Mel tensor shape:", mel)
    break

# Load the preprocessed tensor
# tensor_path = 'data/mel/chinese/chinese0.npy'
# input_tensor = preprocessing.read_spectrogram(tensor_path)
dataset = MelSpectrogramDataset(root_dir='data')
train_dataset, test_dataset = create_train_test_split(dataset, test_size=0.2)

# Creating DataLoaders for train and test datasets
from torch.utils.data import DataLoader

batch_size = 32
train_loader = DataLoader(dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset, batch_size = batch_size, shuffle = False)

# For testing
for label, mel in train_loader:
    print("Label (one-hot):", label)
    print("Mel tensor shape:", mel.shape)
    break

input_tensor = train_loader
print(input_tensor.shape)

# Ensure the tensor has the correct shape
if input_tensor.dim() == 3:  # [channels, height, width]
    input_tensor = input_tensor.unsqueeze(0)  # [1, channels, height, width]
    
# Expand the single channel to three channels
input_tensor = input_tensor.repeat(1, 3, 1, 1)  # Resulting shape: [1, 3, 128, 241]
# Wrap in Variable (optional)
input_var = Variable(input_tensor, requires_grad=False)

# Move to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_var = input_var.to(device)

# Perform forward pass
output_logits = model1(input_var)

Train Label (one-hot): tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]])
Train Mel tensor shape: tensor([[[[-100.0000, -100.0000, -100.0000,  ..., -100.0000, -100.0000,
           -100.0000],
          [-100.0000, -100.0000,  -72.4546,  ...,  -13.6483,  -14.6870,
            -16.4243],
          [-100.0000, -100.0000,  -65.1433,  ...,   -6.3371,   -7.3758,
             -9.1131],
          ...,
          [-100.0000,  -99.3428,  -74.4202,  ...,  -65.3932,  -65.2922,
            -57.0982],
          [-100.0000, -100.0000,  -78.4718,  ...,  -61.2901,  -58.0651,
            -56.7988],
          [-100.0000, -100.0000,  -70.8652,  ...,  -57.5549,  -56.7180,
            -44.1332]]],


        [[[-100.0000, -100.0000, -100.0000,  ..., -100.0000, -100.0000,
           -100.0000],
          [ -17.0062,  -23.4803,  -20.2035,  ...,  -17.9718,  -20.3607,
             -5.7881],
          [  -9.6949,  -16.1691,  -12.8922,  ...,  -10.6605,  -13.049

NameError: name 'dataloader' is not defined